In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import os

In [2]:
links = [
"https://www.ilifepost.com/category/%E4%BA%BA%E7%94%9F%E7%99%BE%E6%80%81/",
"https://www.ilifepost.com/category/%E5%85%BB%E7%94%9F/%E4%BF%9D%E5%81%A5/",
"https://www.ilifepost.com/category/%E5%85%BB%E7%94%9F/%E6%95%91%E5%91%BD/",
"https://www.ilifepost.com/category/%E4%BC%91%E9%97%B2/%E7%8E%A9%E4%B9%90/", 
"https://www.ilifepost.com/category/%E4%BC%91%E9%97%B2/%E6%97%85%E6%B8%B8/", 
"https://www.ilifepost.com/category/%E6%B0%91%E4%BF%97/", 
"https://www.ilifepost.com/category/%E9%A3%8E%E6%B0%B4/%E5%91%BD%E7%90%86%E9%A3%8E%E6%B0%B4/",
"https://www.ilifepost.com/category/%E8%83%BD%E9%87%8F/",
"https://www.ilifepost.com/category/%E4%B8%87%E5%AD%97/%E4%B8%AD%E5%A5%96/",
"https://www.ilifepost.com/category/%E5%95%86%E8%AE%AF/"]
    
#link not including pleasant reading category and video, and also one page articles semua belum

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [33]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            if r.status_code == 508:
                print(r.status_code)
                print('error in link, retrying... ', url)
                time.sleep(20)
                continue
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "lxml")

    block = soup.find_all('div',attrs = {"class":"article-div"})
    
    if block is None:
        return

    for link in soup.find_all('div',attrs = {"class":"article-div"}):
        
        href = link.find('a').get('href')

        hrefs.append(href)
    

In [ ]:
max_worker = 10

for url in links:
    
    print(url)
    r = requests.get(url,headers = headers)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page/')[1].replace('/','')) for a_ in pgno])
    print(max_page)
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}/page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()


In [36]:
hrefs = list(set(hrefs))

In [7]:
len(hrefs)

27203

In [38]:
with open('ilifepost-link.json', 'a') as f:
    json.dump(hrefs, f)

In [3]:
with open('ilifepost-link.json') as fopen:
    hrefs = json.load(fopen)

In [14]:
def process_url(x):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="entry-title").text
        h = soup.find('div', attrs = {"class":"entry-content clearfix"})
        p = [p.text for p in h.find_all("p") if p.text]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    return data

In [ ]:
max_workers = 10

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('ilifepost.jsonl', 'a',encoding='utf8') as f:
                json.dump(result, f,ensure_ascii=False)
                f.write('\n')